In [1]:
import lammps
import pymc3 as pm
import arviz as az
import numpy as np
import glob
from copy import deepcopy
import matplotlib.pyplot as plt

from lammps_data import LammpsData
from pymatgen.io.vasp import Poscar

In [2]:
params = {}
params['core_shell'] = { 'Li': False, 'Ni': False, 'O': True }
params['charges'] = {'Li': +1.0,
                     'Ni': +3.0,
                     'O': {'core':  +0.960,
                           'shell': -2.960}}
params['masses'] = {'Li': 6.941,
                    'Ni': 58.6934,
                    'O': {'core': 15.0,
                          'shell': 1.0} }

params['bpp'] = {'Li-O' : [632.1018, 0.2906, 0.0],
                 'Ni-O' : [1582.5000, 0.2882, 0.000],
                 'O-O'  : [22764.3000, 0.1490, 21.7]}

bpp = deepcopy(params['bpp'])



poscar = Poscar.from_file('POSCAR1')
my_structure = poscar.structure
lammps_data = LammpsData.from_structure(my_structure, params)

Found elements: ['Li', 'Ni', 'O']


In [3]:
for pot in lammps_data.potentials:
    print(pot.potential_string())
#     pot.update('name', 10.0)
#     print(pot.a)

pair_coeff 1 4 632.1018 0.2906 0.0
pair_coeff 2 4 1582.5 0.2882 0.0
pair_coeff 4 4 22764.3 0.149 21.7


In [4]:
# for pot in lammps_data.potentials:
#     pot.a.distribution(stand_dev = 100)

In [43]:
with pm.Model() as model:
    my_dict = {}
    for pot in lammps_data.potentials:
        name = '{}'.format(pot.a.label_string)
        my_dict[name] = pot.a.distribution(stand_dev = 100)
        name = '{}'.format(pot.rho.label_string)
        my_dict[name] = pot.rho.distribution(stand_dev = 0.05)
        name = '{}'.format(pot.c.label_string)
        my_dict[name] = pot.c.distribution(stand_dev = 0.01)
#         exec("%s = %s" % ("{}".format(pot.a.label_string), distribution))

In [44]:
my_dict

{'Li_O_a': Li_O_a,
 'Li_O_rho': Li_O_rho,
 'Li_O_c': Li_O_c,
 'Ni_O_a': Ni_O_a,
 'Ni_O_rho': Ni_O_rho,
 'Ni_O_c': Ni_O_c,
 'O_O_a': O_O_a,
 'O_O_rho': O_O_rho,
 'O_O_c': O_O_c}

In [47]:
my_dict['O_O_a']

O_O_a

In [48]:
type(my_dict['O_O_a'])

pymc3.model.FreeRV